In [11]:
# datetime per hour, fire, other details such as tree or metadata, polygon

# create geopandas
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from datetime import datetime, timedelta

# Define the bounding box (frame) for the grid
# (min_lon, min_lat, max_lon, max_lat)
bounding_box = (20.0, 34.0, 29.0, 42.0)  # Example for area of Greece

# Spacing between polygons (0.25 degrees)
spacing = 0.25
sh = spacing/2 # halved area

# Create the polygons grid within the bounding box
def create_grid(min_lon, min_lat, max_lon, max_lat, spacing):
    gids = []
    gid = 1
    polygons = []
    lons = []
    lats = []
    x = min_lon
    while x < max_lon:
        y = min_lat
        while y < max_lat:
            # Create a square polygon
            poly = Polygon([
                (x-sh, y-sh), 
                (x+sh, y-sh), 
                (x+sh, y+sh), 
                (x-sh, y+sh)
            ])
            gids.append(gid)
            polygons.append(poly)
            lons.append(x)
            lats.append(y)

            y += spacing
            gid += 1
        x += spacing
    return gids, polygons, lons, lats

# Generate the grid of polygons
gids, polygons, lons, lats = create_grid(*bounding_box, spacing)



In [12]:
print(len(gids))

1152


In [64]:
# Create a time range with hourly intervals from June 2023 to August 2023
start_time = datetime(2023, 6, 1, 0, 0)
end_time = datetime(2023, 6, 7, 22, 59)
start_time_range = pd.date_range(start_time, end_time, freq='1h', inclusive="left")
end_time_range = pd.date_range(datetime(2023, 6, 1, 1, 0), datetime(2023, 6, 7, 23, 59), freq="1h", inclusive="left")

print(start_time_range.__len__)

<bound method Index.__len__ of DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 01:00:00',
               '2023-06-01 02:00:00', '2023-06-01 03:00:00',
               '2023-06-01 04:00:00', '2023-06-01 05:00:00',
               '2023-06-01 06:00:00', '2023-06-01 07:00:00',
               '2023-06-01 08:00:00', '2023-06-01 09:00:00',
               ...
               '2023-06-07 13:00:00', '2023-06-07 14:00:00',
               '2023-06-07 15:00:00', '2023-06-07 16:00:00',
               '2023-06-07 17:00:00', '2023-06-07 18:00:00',
               '2023-06-07 19:00:00', '2023-06-07 20:00:00',
               '2023-06-07 21:00:00', '2023-06-07 22:00:00'],
              dtype='datetime64[ns]', length=167, freq='h')>


In [65]:
# create polygon
rows = []

# Iterate over each polygon and each timestamp
for j in range(len(start_time_range)):
    for i, gid in enumerate(gids):
        rows.append({
            'id': gid,
            'start_time': start_time_range[j],
            'end_time': end_time_range[j],
            'lon': lons[i],  # Add longitude of bottom-left corner for reference
            'lat': lats[i],  # Add latitude of bottom-left corner for reference
        })

In [66]:
poly_rows = []

for i, zipped in enumerate(zip(gids, polygons)):
    gid, poly = zipped
    poly_rows.append({
        "id": gid,
        "geometry": poly
    })

In [67]:
short_df = pd.DataFrame(rows)
grid_gdf = gpd.GeoDataFrame(poly_rows, crs="EPSG:4326")

# Save to a file (GeoPackage or any format)
grid_gdf.to_file('polygons.gpkg', driver='GPKG')

# Print some of the data for verification
print(grid_gdf.head())
grid_gdf.describe()

   id                                           geometry
0   1  POLYGON ((19.87500 33.87500, 20.12500 33.87500...
1   2  POLYGON ((19.87500 34.12500, 20.12500 34.12500...
2   3  POLYGON ((19.87500 34.37500, 20.12500 34.37500...
3   4  POLYGON ((19.87500 34.62500, 20.12500 34.62500...
4   5  POLYGON ((19.87500 34.87500, 20.12500 34.87500...


,id
count,1152.000000
mean,576.500000
std,332.698061
min,1.000000
25%,288.750000
50%,576.500000
75%,864.250000
max,1152.000000


In [43]:
grid_gdf.to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"id": 1}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 33.875], [20.125, 33.875], [20.125, 34.125], [19.875, 34.125], [19.875, 33.875]]]}}, {"id": "1", "type": "Feature", "properties": {"id": 2}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.125], [20.125, 34.125], [20.125, 34.375], [19.875, 34.375], [19.875, 34.125]]]}}, {"id": "2", "type": "Feature", "properties": {"id": 3}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.375], [20.125, 34.375], [20.125, 34.625], [19.875, 34.625], [19.875, 34.375]]]}}, {"id": "3", "type": "Feature", "properties": {"id": 4}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.625], [20.125, 34.625], [20.125, 34.875], [19.875, 34.875], [19.875, 34.625]]]}}, {"id": "4", "type": "Feature", "properties": {"id": 5}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 34.875], [20.125, 34.875], [20.125, 35.125], [

In [68]:
short_df.head()
short_df.describe()

,id,start_time,end_time,lon,lat
count,192384.000000,192384,192384,192384.00000,192384.000000
mean,576.500000,2023-06-04 10:59:59.999999744,2023-06-04 12:00:00,24.37500,37.875000
min,1.000000,2023-06-01 00:00:00,2023-06-01 01:00:00,20.00000,34.000000
25%,288.750000,2023-06-02 17:00:00,2023-06-02 18:00:00,22.18750,35.937500
50%,576.500000,2023-06-04 11:00:00,2023-06-04 12:00:00,24.37500,37.875000
75%,864.250000,2023-06-06 05:00:00,2023-06-06 06:00:00,26.56250,39.812500
max,1152.000000,2023-06-07 22:00:00,2023-06-07 23:00:00,28.75000,41.750000
std,332.554494,NaN,NaN,2.59708,2.308279


In [69]:
# create target area to filter grid

geojson_path = '/Users/jw/VSCProjects/hackathon_stellarSustainers/src/world_HD.geo.json'

# Load the GeoJSON into a GeoDataFrame
gdf = gpd.read_file(geojson_path)

# Assuming your GeoJSON contains MultiPolygons or polygons, 
# you can filter for MultiPolygons if needed
multipolygons = gdf[gdf.geometry.type == 'MultiPolygon']

gdf.head()

gdf_greece = gdf[gdf['sovereignt'] == 'Greece']


In [46]:
gdf_greece

,featurecla,scalerank,labelrank,sovereignt,sov_a3,adm0_dif,level,type,tlc,admin,...,fclass_id,fclass_pl,fclass_gr,fclass_it,fclass_nl,fclass_se,fclass_bd,fclass_ua,filename,geometry
196,Admin-0 country,0,3,Greece,GRC,0,2,Sovereign country,1,Greece,...,None,None,None,None,None,None,None,None,GRC.geojson,"MULTIPOLYGON (((20.96526 40.84939, 21.11213 40..."


In [47]:
gdf_greece.to_json()
grid_gdf

,id,geometry
0,1,"POLYGON ((19.87500 33.87500, 20.12500 33.87500..."
1,2,"POLYGON ((19.87500 34.12500, 20.12500 34.12500..."
2,3,"POLYGON ((19.87500 34.37500, 20.12500 34.37500..."
3,4,"POLYGON ((19.87500 34.62500, 20.12500 34.62500..."
4,5,"POLYGON ((19.87500 34.87500, 20.12500 34.87500..."
...,...,...
1147,1148,"POLYGON ((28.62500 40.62500, 28.87500 40.62500..."
1148,1149,"POLYGON ((28.62500 40.87500, 28.87500 40.87500..."
1149,1150,"POLYGON ((28.62500 41.12500, 28.87500 41.12500..."
1150,1151,"POLYGON ((28.62500 41.37500, 28.87500 41.37500..."


In [70]:
multipolygon = gdf_greece.geometry.iloc[0]

# Filter the polygons in gdf_polygons that intersect with the MultiPolygon
filtered_polygons = grid_gdf[grid_gdf.intersects(multipolygon)]

# Print the filtered GeoDataFrame
print(filtered_polygons)
filtered_polygons.to_json()

        id                                           geometry
21      22  POLYGON ((19.87500 39.12500, 20.12500 39.12500...
22      23  POLYGON ((19.87500 39.37500, 20.12500 39.37500...
23      24  POLYGON ((19.87500 39.62500, 20.12500 39.62500...
49      50  POLYGON ((20.12500 38.12500, 20.37500 38.12500...
53      54  POLYGON ((20.12500 39.12500, 20.37500 39.12500...
...    ...                                                ...
1032  1033  POLYGON ((27.87500 35.87500, 28.12500 35.87500...
1033  1034  POLYGON ((27.87500 36.12500, 28.12500 36.12500...
1034  1035  POLYGON ((27.87500 36.37500, 28.12500 36.37500...
1065  1066  POLYGON ((28.12500 36.12500, 28.37500 36.12500...
1066  1067  POLYGON ((28.12500 36.37500, 28.37500 36.37500...

[440 rows x 2 columns]


'{"type": "FeatureCollection", "features": [{"id": "21", "type": "Feature", "properties": {"id": 22}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.125], [20.125, 39.125], [20.125, 39.375], [19.875, 39.375], [19.875, 39.125]]]}}, {"id": "22", "type": "Feature", "properties": {"id": 23}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.375], [20.125, 39.375], [20.125, 39.625], [19.875, 39.625], [19.875, 39.375]]]}}, {"id": "23", "type": "Feature", "properties": {"id": 24}, "geometry": {"type": "Polygon", "coordinates": [[[19.875, 39.625], [20.125, 39.625], [20.125, 39.875], [19.875, 39.875], [19.875, 39.625]]]}}, {"id": "49", "type": "Feature", "properties": {"id": 50}, "geometry": {"type": "Polygon", "coordinates": [[[20.125, 38.125], [20.375, 38.125], [20.375, 38.375], [20.125, 38.375], [20.125, 38.125]]]}}, {"id": "53", "type": "Feature", "properties": {"id": 54}, "geometry": {"type": "Polygon", "coordinates": [[[20.125, 39.125], [20.375, 39.125], [20.375, 

In [75]:
# only have time and polygons that exist in Greece

# create polygon
rows = []

# Iterate over each polygon and each timestamp
for j in range(len(start_time_range)):
    for i, frow in filtered_polygons.iterrows():
        rows.append({
            'id': frow["id"],
            'start_time': start_time_range[j],
            'end_time': end_time_range[j],
            'lon': lons[i],  # Add longitude of bottom-left corner for reference
            'lat': lats[i],  # Add latitude of bottom-left corner for reference
        })

hour_df = pd.DataFrame(rows)
hour_df.head()

,id,start_time,end_time,lon,lat
0,22,2023-06-01,2023-06-01 01:00:00,20.00,39.25
1,23,2023-06-01,2023-06-01 01:00:00,20.00,39.50
2,24,2023-06-01,2023-06-01 01:00:00,20.00,39.75
3,50,2023-06-01,2023-06-01 01:00:00,20.25,38.25
4,54,2023-06-01,2023-06-01 01:00:00,20.25,39.25


In [71]:
# Load the GeoJSON and grid GeoDataFrames
fire_gdf = gpd.read_file('/Users/jw/VSCProjects/hackathon_stellarSustainers/src/transformed_file.geojson')
filtered_grid = filtered_polygons

matches = []

for i, grid_poly in fire_gdf.iterrows():
    for j, comp_poly in filtered_grid.iterrows():
        # Step 3: Check for intersections
        if grid_poly.geometry.intersects(comp_poly.geometry):
            matches.append({"fire": grid_poly["id"], "polygon":comp_poly["id"]})


/Users/jw/anaconda3/envs/cassini/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/Users/jw/anaconda3/envs/cassini/lib/python3.10/site-packages/geopandas/io/file.py:403: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)
/Users/jw/anaconda3/envs/cassini/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/Users/jw/anaconda3/envs/cassini/lib/python3.10/site-packages/geopandas/io/file.py:403: Futur

In [50]:
matches

[{'fire': 14843, 'polygon': 912},
 {'fire': 14389, 'polygon': 210},
 {'fire': 14329, 'polygon': 311},
 {'fire': 15314, 'polygon': 113},
 {'fire': 15002, 'polygon': 444},
 {'fire': 14324, 'polygon': 444},
 {'fire': 14946, 'polygon': 444},
 {'fire': 14676, 'polygon': 665},
 {'fire': 15324, 'polygon': 665},
 {'fire': 15103, 'polygon': 537},
 {'fire': 14219, 'polygon': 590},
 {'fire': 15269, 'polygon': 790},
 {'fire': 15280, 'polygon': 790},
 {'fire': 15422, 'polygon': 909},
 {'fire': 15437, 'polygon': 614},
 {'fire': 15471, 'polygon': 342},
 {'fire': 15492, 'polygon': 790},
 {'fire': 15496, 'polygon': 279},
 {'fire': 15665, 'polygon': 531},
 {'fire': 15819, 'polygon': 148},
 {'fire': 15896, 'polygon': 121},
 {'fire': 15911, 'polygon': 339},
 {'fire': 16011, 'polygon': 309},
 {'fire': 16167, 'polygon': 790},
 {'fire': 16195, 'polygon': 790},
 {'fire': 16211, 'polygon': 86},
 {'fire': 16252, 'polygon': 402},
 {'fire': 16275, 'polygon': 510},
 {'fire': 16361, 'polygon': 467},
 {'fire': 16361

In [51]:
print(len(matches))
filtered_grid.loc[filtered_grid['id'] == 594,'geometry'].to_json()

1228


'{"type": "FeatureCollection", "features": [{"id": "593", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[24.375, 38.125], [24.625, 38.125], [24.625, 38.375], [24.375, 38.375], [24.375, 38.125]]]}, "bbox": [24.375, 38.125, 24.625, 38.375]}], "bbox": [24.375, 38.125, 24.625, 38.375]}'

In [52]:
fire_gdf.loc[fire_gdf['id'] == 223671,'geometry'].to_json()

'{"type": "FeatureCollection", "features": [{"id": "888", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[24.58939335304745, 38.140701237777805], [24.58933579767129, 38.14075441671835], [24.589301790027775, 38.14082761608412], [24.58929133017141, 38.140920835884245], [24.589304418219584, 38.141034076132165], [24.5893410542942, 38.141167336790154], [24.589401238615526, 38.1413206178284], [24.589484971419537, 38.141493919164326], [24.589592252971556, 38.14168724072473], [24.58972308362551, 38.14190058239986], [24.589841902545732, 38.14209646553153], [24.58994870951788, 38.14227489018385], [24.590043504326456, 38.142435856417926], [24.59012628679034, 38.1425793642956], [24.590197056754963, 38.14270541384387], [24.59025581407241, 38.14281400512228], [24.590302558630583, 38.1429051381449], [24.590337290324026, 38.14297881295837], [24.590368919081694, 38.14304600656862], [24.590397444883585, 38.143106718990005], [24.590422867706028, 38.14316095020

In [77]:
fire_gdf.head()

,id,initialdate,finaldate,area_ha,iso2,iso3,country,admlvl1,admlvl2,admlvl3,...,other_natural_percent,agriculture_percent,artificial_percent,other_percent,natura2k_percent,noneu,updated,area_code,eu_area,geometry
0,14843,2018-09-09T00:00:00,2018-09-10T00:00:00+00:00,147,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Βόρειο Αιγαίο,"Ικαρία, Σάμος",...,0.000000,35.135135,0.000000,0.0,0.000000,False,2022-01-26T12:15:03.089000,EU,EU,"MULTIPOLYGON (((26.920 37.766, 26.920 37.766, ..."
1,14389,2018-09-21T00:00:00,2018-09-21T00:00:00+00:00,75,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Δυτική Ελλάδα,Αχαΐα,...,97.297297,2.702703,0.000000,0.0,66.535296,False,2022-01-26T12:15:03.089000,EU,EU,"MULTIPOLYGON (((21.408 38.143, 21.408 38.143, ..."
2,14329,2018-09-23T00:00:00,2018-09-23T00:00:00+00:00,192,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Θεσσαλία,Λάρισα,...,54.736842,45.263158,0.000000,0.0,85.111159,False,2022-01-26T12:15:03.089000,EU,EU,"MULTIPOLYGON (((22.249 39.476, 22.249 39.476, ..."
3,15314,2018-09-27T00:00:00,2018-09-27T00:00:00+00:00,270,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Ιόνια Νησιά,"Ιθάκη, Κεφαλληνία",...,18.888889,67.037037,0.740741,0.0,0.000000,False,2022-01-26T12:15:03.089000,EU,EU,"MULTIPOLYGON (((20.751 38.074, 20.751 38.074, ..."
4,15002,2018-10-09T00:00:00,2018-10-09T00:00:00+00:00,51,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,Κεντρική Μακεδονία,Θεσσαλονίκη,...,96.000000,0.000000,0.000000,0.0,100.000000,False,2022-01-26T12:15:03.089000,EU,EU,"MULTIPOLYGON (((23.324 40.731, 23.324 40.731, ..."


In [78]:
fire_gdf['start_date'] = pd.to_datetime(fire_gdf['initialdate'], format='mixed')
fire_gdf['end_date'] = pd.to_datetime(fire_gdf['finaldate'], format='mixed')
# Step 3: Filter the DataFrame based on the date column
filtered_fire = fire_gdf[(fire_gdf['start_date'] >= datetime(2024, 8, 1, 0, 0)) & (fire_gdf['start_date'] <= datetime(2025, 8, 1, 0, 0))]

In [79]:
filtered_fire

,id,initialdate,finaldate,area_ha,iso2,iso3,country,admlvl1,admlvl2,admlvl3,...,artificial_percent,other_percent,natura2k_percent,noneu,updated,area_code,eu_area,geometry,start_date,end_date
1011,242533,2024-08-29T00:54:00,2024-08-29T00:54:00+00:00,19,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,Δυτική Μακεδονία,"Γρεβενά, Κοζάνη",...,0.000000,0.0,0.000000,False,2024-09-13T07:41:17.913000,EU,EU,"MULTIPOLYGON (((22.046 40.403, 22.046 40.403, ...",2024-08-29 00:54:00,2024-08-29 00:54:00+00:00
1053,236075,2024-08-01T00:33:00,2024-08-01T00:58:00+00:00,61,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Κρήτη,Ρέθυμνο,...,0.000000,0.0,98.950177,False,2024-08-02T14:03:42.553001,EU,EU,"MULTIPOLYGON (((24.403 35.225, 24.402 35.225, ...",2024-08-01 00:33:00,2024-08-01 00:58:00+00:00
1054,236700,2024-08-02T10:43:00,2024-08-02T10:43:00+00:00,23,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,Κεντρική Μακεδονία,Θεσσαλονίκη,...,0.000000,0.0,82.950816,False,2024-08-07T09:14:01.982000,EU,EU,"MULTIPOLYGON (((23.057 40.780, 23.057 40.780, ...",2024-08-02 10:43:00,2024-08-02 10:43:00+00:00
1055,239812,2024-08-25T00:05:00,2024-08-25T00:30:00+00:00,1,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Θεσσαλία,Λάρισα,...,0.000000,0.0,0.000000,False,2024-08-28T15:07:06.644000,EU,EU,"MULTIPOLYGON (((22.422 39.214, 22.422 39.214, ...",2024-08-25 00:05:00,2024-08-25 00:30:00+00:00
1056,236134,2024-08-01T10:11:00,2024-08-01T11:02:00+00:00,26,EL,GRC,Ελλάδα,Αττική,Aττική,Ανατολική Αττική,...,0.000000,0.0,0.000000,False,2024-08-02T16:03:20.283001,EU,EU,"MULTIPOLYGON (((23.875 37.828, 23.875 37.829, ...",2024-08-01 10:11:00,2024-08-01 11:02:00+00:00
1058,236477,2024-08-03T10:24:00,2024-08-03T10:24:00+00:00,1,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Θεσσαλία,"Καρδίτσα, Τρίκαλα",...,NaN,NaN,0.000000,False,2024-08-06T07:21:47.333000,EU,EU,"MULTIPOLYGON (((21.741 39.246, 21.741 39.246, ...",2024-08-03 10:24:00,2024-08-03 10:24:00+00:00
1060,236396,2024-08-04T11:47:00,2024-08-04T11:57:00+00:00,106,EL,GRC,Ελλάδα,Κεντρική Ελλάδα,Στερεά Ελλάδα,Φθιώτιδα,...,0.000000,0.0,0.000000,False,2024-08-06T13:41:05.532000,EU,EU,"MULTIPOLYGON (((22.288 39.041, 22.287 39.040, ...",2024-08-04 11:47:00,2024-08-04 11:57:00+00:00
1061,236779,2024-08-08T00:01:00,2024-08-08T00:50:00+00:00,16,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Νότιο Αιγαίο,"Κάλυμνος, Κάρπαθος – Ηρωική Νήσος Κάσος, Κως, ...",...,0.000000,0.0,0.000000,False,2024-08-08T06:56:52.117001,EU,EU,"MULTIPOLYGON (((27.016 36.956, 27.016 36.956, ...",2024-08-08 00:01:00,2024-08-08 00:50:00+00:00
1062,236775,2024-08-03T11:17:00,2024-08-04T00:52:00+00:00,301,EL,GRC,Ελλάδα,Βόρεια Ελλάδα,"Aνατολική Μακεδονία, Θράκη",Ροδόπη,...,0.000000,0.0,100.000000,False,2024-08-08T08:43:15.434000,EU,EU,"MULTIPOLYGON (((25.795 41.129, 25.795 41.129, ...",2024-08-03 11:17:00,2024-08-04 00:52:00+00:00
1063,237138,2024-08-07T11:40:00,2024-08-09T01:00:00+00:00,980,EL,GRC,Ελλάδα,"Νησιά Αιγαίου, Κρήτη",Κρήτη,Ρέθυμνο,...,0.000000,0.0,0.458373,False,2024-08-09T13:25:57.654999,EU,EU,"MULTIPOLYGON (((24.706 35.162, 24.706 35.162, ...",2024-08-07 11:40:00,2024-08-09 01:00:00+00:00


In [82]:
def categorize(row, fire_row):
    start_date_fire, end_date_fire = fire_row['start_date'], fire_row['start_date']

    if row['age'] < 30 and row['income'] < 60000:
        return 'Young and Low Income'
    elif row['age'] >= 30 and row['income'] < 70000:
        return 'Mature and Low Income'
    elif row['age'] >= 30 and row['income'] >= 70000:
        return 'Mature and High Income'
    else:
        return 0

# Apply the function to each row
hour_df['category'] = hour_df.apply(categorize, args=(argsaxis=1)
for match in matches:
    fire_id = match["fire"]
    grid_id = match["polygon"]
    fire_row = fire_gdf[fire_gdf["id"] == fire_id]

    
    # print(match, row)
    hour_df["fire"]
    hour_df[(hour_df['start_time'] <= start_date_fire) & (fire_gdf['start_date'] <= end_date_fire)]
# Step 3: Filter the DataFrame based on the date column
# hour_df["fire"] = fire_gdf[(fire_gdf['start_date'] >= datetime(2024, 8, 1, 0, 0)) & (fire_gdf['start_date'] <= datetime(2025, 8, 1, 0, 0))]

{'fire': 14843, 'polygon': 912}       id          initialdate                  finaldate  area_ha iso2 iso3  \
0  14843  2018-09-09T00:00:00  2018-09-10T00:00:00+00:00      147   EL  GRC   

   country               admlvl1        admlvl2        admlvl3  ...  \
0  Ελλάδα   Νησιά Αιγαίου, Κρήτη  Βόρειο Αιγαίο  Ικαρία, Σάμος  ...   

  artificial_percent other_percent  natura2k_percent  noneu  \
0                0.0           0.0               0.0  False   

                      updated  area_code  eu_area  \
0  2022-01-26T12:15:03.089000         EU       EU   

                                            geometry  start_date  \
0  MULTIPOLYGON (((26.920 37.766, 26.920 37.766, ...  2018-09-09   

                   end_date  
0 2018-09-10 00:00:00+00:00  

[1 rows x 29 columns]
{'fire': 14389, 'polygon': 210}       id          initialdate                  finaldate  area_ha iso2 iso3  \
1  14389  2018-09-21T00:00:00  2018-09-21T00:00:00+00:00       75   EL  GRC   

   country          a